# Sprint 3

In de [derde sprint](https://trello.com/b/EUxZhLuE/sprint-3-milestone-1-27-nov) werken we aan de volgende taken:

1. [Helper Functions](#1.-Helper-Functions)
2. [Top Layer van het Systeem](#2.-Top-Layer-van-het-Systeem) ~Jim
3. [Feature score berekenen van woorden, feature vectors maken](#3.-Feature-score-berekenen-van-woorden,-feature-vectors-maken) ~Stefan
4. [Fuzzy Logic Toolbox implementeren](#4.-Fuzzy-Logic-Toolbox-implementeren) ~Peter

### 1. Helper Functions

In [Sprint 2](http://localhost:8888/notebooks/Sprint%202.ipynb) hebben we methodes gemaakt om woorden te tellen en een intersectie tussen woordenlijsten uit te voeren. Deze methodes kunnen worden gebruikt bij de taken.

In [57]:
import csv
import nltk
import string
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def tokenize(body):
    tokens = word_tokenize(body)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    return words

def read_txt(filepath):
    with open(filepath, 'r') as file:
        body = t.read()
    return tokenize(body)     
    
def read_csv(filepath):
    with open(filepath, 'r') as c:
        reader = csv.reader(c, delimiter=',')
        for row in reader:
            return row

def generate_csv_from_array(filename, array):
    with open("res/" + filename + ".csv", 'w', newline='') as c:
        writer = csv.writer(c, delimiter=',')
        writer.writerow(array)
        
def intersection(array1, array2):
    """Returns a generator, use next(generator)"""
    return (i for i in array1 if i in array2)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2. Top Layer van het Systeem
    ~Jim Kamans

Er moet een systeem geschreven worden (gedurende alle sprints) die de basis vormt voor het classificeren van meerdere emails, waarbij een overzicht van de classificatie en analyse van de correct en fout geclassificeerde emails wordt getoond.

#### 2.1 Inlezen meerdere emails uit folder

We zoeken in het path van de dataset naar alle emails van alle employees (in get_emails()). Deze lezen we in met read_email(), welke een tokenized list returned.  

In [65]:
from os import listdir
from os.path import isfile, join

def read_email(filepath):
    with open(filepath, 'r') as file:
        text = ""
        reading = False
        
        # start reading email after the first blank line (so crap like x-header etc is gone)
        # stop reading when a "forwarded part" is encountered
        for line in file:
            if reading:
                if "---- forwarded" in line.lower(): 
                    break
                text += line
            else:
                if line in ['\n','\r\n']:
                    reading = True
    return tokenize(text)

def get_emails():
    path_of_dataset = "../_email/enron_mail_20150507/maildir"
    mail_folder = "all_documents"
    employees = listdir(path_of_dataset)  
    emails = []
    
    for employee in employees[:10]:
#     for employee in employees:
        path_to_emails = path_of_dataset + "/" + employee + "/" + mail_folder
        try:
            emails += [(path_to_emails + "/" + f) for f in listdir(path_to_emails) if f.endswith(".")]
        except:
            print("No {} folder:".format(mail_folder), employee)
            pass

    return emails

emails = get_emails()
for email in emails[5:15]:
    tokens = read_txt(email)
    print(tokens)

# read_txt("../_email/enron_mail_20150507/maildir/allen-p/_sent_mail/2")

No all_documents folder: blair-l
['paula', 'million', 'fine', 'phillip']
[]
[]
[]
['brenda', 'pleas', 'use', 'second', 'check', 'octob', 'payment', 'alreadi', 'toss', 'let', 'know', 'mail', 'anoth', 'phillip']
['think', 'fletch', 'good', 'cpa', 'still']
['brenda', 'pleas', 'use', 'second', 'check', 'octob', 'payment', 'copi', 'origin', 'deal', 'want', 'fax', 'phillip']
['sacramento', 'correspond', 'exit', 'news', 'confer', 'gov', 'davi', 'ferc', 'chair', 'hoecker', 'doe', 'sectretari', 'richardson', 'other', 'outlin', 'sever', 'emerg', 'measur', 'includ', 'westwid', 'price', 'cap', 'soon', 'report', 'file', 'send', 'attent', 'expect', 'around', 'pm']
[]
['lian', 'discuss', 'yesterday', 'concern', 'attempt', 'manipul', 'el', 'paso', 'san', 'juan', 'monthli', 'index', 'singl', 'buyer', 'enter', 'marketplac', 'septemb', 'paid', 'market', 'price', 'san', 'juan', 'ga', 'intent', 'distort', 'index', 'time', 'trade', 'offer', 'physic', 'ga', 'significantli', 'cent', 'lower', 'price', 'bypass'

['outlook',
 'migrat',
 'team',
 'enron',
 'pm',
 'wilchynskihrcorpenron',
 'enron',
 'cindi',
 'r',
 'wardnaenron',
 'enron',
 'j',
 'ann',
 'hillcorpenron',
 'enron',
 'sonja',
 'gallowaycorpenron',
 'enron',
 'bilal',
 'bajwan',
 'aenron',
 'enron',
 'binh',
 'phamhouect',
 'ect',
 'bradley',
 'jonesenron',
 'enronxg',
 'bruce',
 'millscorpenron',
 'enron',
 'chanc',
 'rabonenron',
 'enronxg',
 'chuck',
 'amesnaenron',
 'enron',
 'david',
 'baumbachhouect',
 'ect',
 'jad',
 'doanenron',
 'enronxg',
 'oneal',
 'win',
 'freehouect',
 'ect',
 'phillip',
 'lovehouect',
 'ect',
 'sladanaanna',
 'kulicenron',
 'enro',
 'nxgate',
 'victor',
 'guggenheimhouect',
 'ect',
 'alejandra',
 'chaveznaenron',
 'enron',
 'ann',
 'e',
 'bikeenron',
 'enronxg',
 'carol',
 'franknaenron',
 'enron',
 'darron',
 'c',
 'gironhouec',
 'ect',
 'elizabeth',
 'l',
 'hernandezhouect',
 'ect',
 'elizabeth',
 'shimcorpenron',
 'enron',
 'jeff',
 'royedcorpenron',
 'enron',
 'kam',
 'keiserhouect',
 'ect',
 'kima

### 3. Feature score berekenen van woorden, feature vectors maken

    ~Stefan Schenk
    
Woorden in een corpus moeten extra features krijgen, namelijk de scores van de mate waarbij ieder woord bij een feature past. Daarna moet de algehele score van alle woorden bij elkaar worden berekend. Dit zullen de inputs zijn voor het Fuzzy Logic Systeem.

#### 3.1 Generating Word List

We genereren een word_list, die normaal gesproken alle relevante woorden bevat. Daarnaast genereren we twee lijsten: C en M, die de Features voorstellen (lijsten van woorden met dezelfde karakterastieken). 

In [31]:
email = sorted(read_txt('res/email.txt'))
print("EMAIL:", email[:10])
# print("WORD_LIST:", word_list[:10])

# Maakt een feature_list met woorden die met een c beginnen
starts_with_c = [x for x in email if x[0] == 'c']
generate_csv_from_array("starts_with_c", starts_with_c)
print("C:", starts_with_c[:10])

# Maakt een feature_list met woorden die met een m beginnen
starts_with_m = [x for x in email if x[0] == 'm']
generate_csv_from_array("starts_with_m", starts_with_m)
print("M:", starts_with_m[:10])

# Maakt de filter word_list, bestaande uit combinatie van de Features
word_list = starts_with_c + starts_with_m
print("WORD_LIST:", word_list[:10])
generate_csv_from_array("word_list", word_list)

EMAIL: ['abdel', 'abstorsv', 'aid', 'al', 'al', 'al', 'al', 'al', 'al', 'al']
C: ['cairo', 'cairo', 'cairo', 'cairo', 'cairo', 'cairo', 'call', 'call', 'call', 'case']
M: ['mail', 'mail', 'mainli', 'materi', 'meet', 'meet', 'middl', 'minist', 'minist', 'minist']
WORD_LIST: ['cairo', 'cairo', 'cairo', 'cairo', 'cairo', 'cairo', 'call', 'call', 'call', 'case']


#### 3.2 Generating Corpus

In de vorige sprint gebruikten wij de Counter class om distincte woorden te tellen en op te slaan in een dictionary. Jammergenoeg is het lastig om dictionaries uit te breiden met meerdere values.

Numpy heeft een methode "unique" waarmee hetzelfde doel kan worden bereikt. De values en counts worden opgeslagen in een uitbreidbare np.array.

In [32]:
def corpus(email, word_list):
    words = [x for x in intersection(email, word_list)]
    return np.c_[np.unique(words, return_counts=True)]

print(corpus(email, word_list)[:10])

[['cairo' '6']
 ['call' '3']
 ['case' '2']
 ['ceasefir' '2']
 ['center' '1']
 ['cia' '1']
 ['classifi' '1']
 ['clear' '1']
 ['clinton' '1']
 ['comment' '3']]


#### 3.3 Formule Score

Als laatst worden de scores berekend voor alle woorden, voor alle Features.

In [33]:
def rate(corpus, feature_lists):
    c_len = len(corpus)
    c = corpus
    for f in feature_lists:
        c = np.c_[c, np.zeros(c_len)]
        for row in c:
            if (row[0] in f):
                row[-1:] = round(int(row[1]) / c_len, 4)
    return c

rating = rate(
    corpus(email, word_list), 
    [starts_with_c, starts_with_m] )

print("RATING:\n", rating)

RATING:
 [['cairo' '6' '0.2143' '0.0']
 ['call' '3' '0.1071' '0.0']
 ['case' '2' '0.0714' '0.0']
 ['ceasefir' '2' '0.0714' '0.0']
 ['center' '1' '0.0357' '0.0']
 ['cia' '1' '0.0357' '0.0']
 ['classifi' '1' '0.0357' '0.0']
 ['clear' '1' '0.0357' '0.0']
 ['clinton' '1' '0.0357' '0.0']
 ['comment' '3' '0.1071' '0.0']
 ['commun' '1' '0.0357' '0.0']
 ['contribut' '1' '0.0357' '0.0']
 ['cooper' '2' '0.0714' '0.0']
 ['council' '2' '0.0714' '0.0']
 ['countri' '1' '0.0357' '0.0']
 ['creat' '2' '0.0714' '0.0']
 ['crisiss' '1' '0.0357' '0.0']
 ['current' '1' '0.0357' '0.0']
 ['mail' '2' '0.0' '0.0714']
 ['mainli' '1' '0.0' '0.0357']
 ['materi' '1' '0.0' '0.0357']
 ['meet' '2' '0.0' '0.0714']
 ['middl' '1' '0.0' '0.0357']
 ['minist' '5' '0.0' '0.1786']
 ['mission' '2' '0.0' '0.0714']
 ['monitor' '1' '0.0' '0.0357']
 ['monitorspsembassi' '1' '0.0' '0.0357']
 ['muslim' '3' '0.0' '0.1071']]


#### 3.4 Aggregation Scores to Input FIS

De scores worden geaggregeerd zodat zij als input voor het Fuzzy Logic Systeem kunnen dienen.

$max( \: \sum \frac{word_{count}}{word\_list_{count}}, 1.0 \: )$

$
score =
\left\{
	\begin{array}{ll}
		0  & \mbox{if } x \leq 0 \\
        x  & \mbox{if } \{x \mid 0 < x < 1\} \\
		1 & \mbox{if } x \geq 1
	\end{array}
\right.
$

In [34]:
def email_rating(email, feature_lists):
    c = rate(corpus(email, word_list), feature_lists)
    ratings = dict()
    for i in range(len(feature_lists)):
        ratings[i] = min((c[:,i + 2].astype(np.float).sum()), 1.0)
    return ratings
    
inputs = email_rating(email, [starts_with_c, starts_with_m])
print(inputs)

{0: 1.0, 1: 0.67839999999999989}


### 4. Fuzzy Logic Toolbox implementeren

    ~Peter Heemskerk
    
Hieronder de standaard uit het LAB vrekregen code voor een fuzzy logic implementatie. De classes zijn zo gedefinieerd dat je in een aantal stappen een eenvoudige implementatie van een Fuzzy Logic systeem kunt definieren. 

stap a. creeer membership functie mbv. class TriangularMF of class TrapezoidalMF. 
        voorbeeld: 
            triangular_mf = TriangularMF("medium", 150, 250, 350) 
stap b. creer input en output functes met class Input(Variable) of class Output(Variable). 
        voorbeeld: 
            mfs_income = [TrapezoidalMF("low", -100, 0, 200, 400), TriangularMF("medium", 200, 500, 800)]
            income = Input("income", (0, 1000), mfs_income)\
stap c. zet deze inputfuncties en outputfuncies in de lijst met inputs en outputs.
        voorbeeld: 
            inputs = [income, quality]
            output = money
stap d. definieer regels middels class Rule:
        voorbeeld: 
            rule1 = Rule(1, ["low", "amazing"], "and", "low")
stap e. zet alle regels in een class Rulebase: 
        voorbeeld: 
            rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9]
            rulebase = Rulebase(rules).
stap f. je kunt nu firing strenght van deze regel oproepen: 
            rule1.calculate_firing_strength([200, 6.5], inputs), 
        en de totale firing strenghts (!):  
            rulebase.calculate_firing_strengths(datapoint, inputs)
stap g. creeer nu de totale fuzzy logic set middels class Reasoner
        voorbeeld: 
            thinker = Reasoner(rulebase, inputs, output, 201, "som")
stap h. je kunt nu voor een datapunt op je input een crisp/defuzzified output krijgen met de functie: 
            thinker.inference(datapoint)


#### 4.1 Membership Functions

Membership functions are used to fuzzify crisp inputs, representing an item's membership to a class, with 0 meaning no membership and 1 meaning the item is a perfect prototype of a class.

During the Fuzzy Logic crash course we have seen 4 basic types of membership functions: triangular, trapezoidal, gaussian and generalized bell shaped membership functions. Those membership functions are represented in the image:

In [7]:
import math
import numpy as np
from collections import defaultdict, Counter

class TriangularMF:
    """Triangular fuzzy logic membership function class."""
    def __init__(self, name, start, top, end):
        self.name = name
        self.start = start
        self.top = top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start: 
            y = 0
        if x > self.start and x <= self.top: 
            y = (x-self.start)/(self.top-self.start)
        if x > self.top and x <= self.end:
            y = (self.end - x)/(self.end - self.top)
        if x > self.end:
            y = 0
        return y
        
class TrapezoidalMF:
    """Trapezoidal fuzzy logic membership function class."""
    def __init__(self, name, start, left_top, right_top, end):
        self.name = name
        self.start = start
        self.left_top = left_top
        self.right_top = right_top
        self.end = end

    def calculate_membership(self, x):
        if x <= self.start: 
            y = 0
        if x > self.start and x <= self.left_top: 
            y = (x - self.start)/(self.left_top - self.start)
        if x > self.left_top and x <= self.right_top:
            y = 1
        if x > self.right_top and x <= self.end:
            y = (self.end - x)/(self.end - self.right_top)
        if x > self.end:
            y = 0
        return y


In [8]:
# Test your implementation by running the following statements
# Enter your answers in the Google form to check them, round to two decimals

triangular_mf = TriangularMF("medium", 150, 250, 350)
print(triangular_mf.calculate_membership(100))
print(triangular_mf.calculate_membership(249))
print(triangular_mf.calculate_membership(300))

trapezoidal_mf = TrapezoidalMF("bad", 0, 0, 2, 4)
print(trapezoidal_mf.calculate_membership(1.2))
print(trapezoidal_mf.calculate_membership(2.3))
print(trapezoidal_mf.calculate_membership(3.9))

0
0.99
0.5
1
0.8500000000000001
0.050000000000000044


### 4.2. Inputs and output
Theory

The inputs and output of a FLS are represented through linguistic variables, which are variables whose values are words rather than numbers. A value of a linguistic variable is called a linguistic term.

An example of a linguistic variable is 'income' (a variable we're using in the system we're programming), with the linguistic terms 'low', 'medium', 'high'.
Practice

Now we are going to define input and output variables, which are a collection of multiple membership functions.

A variable's membership functions (self.mfs) should be a list of membership functions. Define the input variables income and quality and the output variable money with the name, range and membership functions represented in the image.

In [9]:
class Variable:
    """General class for variables in an FLS."""
    def __init__(self, name, range, mfs):
        self.name = name
        self.range = range
        self.mfs = mfs

    def calculate_memberships(self, x):
        """Test function to check whether
        you put together the right mfs in your variables."""
        return {
            mf.name : mf.calculate_membership(x)
            for mf in self.mfs
        }

    def get_mf_by_name(self, name):
        for mf in self.mfs:
            if mf.name == name:
                return mf

class Input(Variable):
    """Class for input variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "input"

class Output(Variable):
    """Class for output variables, inherits 
    variables and functions from superclass Variable."""
    def __init__(self, name, range, mfs):
        super().__init__(name, range, mfs)
        self.type = "output"

In [10]:
# Input variable for your income
# Your code here
mfs_income = [TrapezoidalMF("low", -100, 0, 200, 400), TriangularMF("medium", 200, 500, 800), TrapezoidalMF("high", 600, 800, 1000, 1200)]
income = Input("income", (0, 1000), mfs_income)

# Input variable for the quality
# Your code here
mfs_quality = [TrapezoidalMF("bad", -1, 0, 2, 4), TriangularMF("okay", 2, 5, 8), TrapezoidalMF("amazing", 6, 8, 10, 12)]
quality = Input("quality", (0, 10), mfs_quality)

# Output variable for the amount of money
# Your code here
mfs_money = [TrapezoidalMF("low", -100, 0, 100, 250), TriangularMF("medium", 150, 250, 350), TrapezoidalMF("high", 250, 400, 500, 600)]
money = Output("money", (0, 500), mfs_money)

inputs = [income, quality]
output = money

In [11]:
# Test your implementation by running the following statements
# Enter your answers in the Google form to check them, round to two decimals

print(income.calculate_memberships(489))
print(quality.calculate_memberships(6))
print(output.calculate_memberships(222))

print(income.name, income.range)
print('inputs:', inputs[0].name, inputs[0].calculate_memberships(489))

{'low': 0, 'medium': 0.9633333333333334, 'high': 0}
{'bad': 0, 'amazing': 0, 'okay': 0.6666666666666666}
{'low': 0.18666666666666668, 'medium': 0.72, 'high': 0}
income (0, 1000)
inputs: income {'low': 0, 'medium': 0.9633333333333334, 'high': 0}


## 4.3 Fuzzy Rules

### Theory

A fuzzy IF-THEN rule is composed of an antecedent and a consequent, like the implications you have seen with propositional and first order logic.
- In the antecedent conditions for input variables are connected with an operator (AND, OR, NOT), e.g. "IF x1 is mf1 AND x2 is mf3", or "IF x1 is mf1 OR x2 is mf2".
- The AND operator represents taking the intersection of fuzzy sets, which can be accomplished by choosing a T-Norm operation, such as minimum.
- The OR operator represents taking the union of fuzzy sets, which can be accomplished by choosing a T-Conorm operation, such as maximum.
- The NOT operator, the complement, is calculated by 1 minus a membership value. For example: NOT x1 is mf1, would be 1 - (membership of x1 to mf1).
- By the use of the AND, NOT and OR we combine the different parts of the antecedent into a single number, which is the firing strength of the antecedent.
- The consequent represents an action that we undertake if the rule fires.

### Practice

We are going to add some simple rules for our FLS: complete rules that do not have mixed operators. Here we represent a rule through 3 variables:
- <b>Antecedent</b>, represented as a list of names of membership functions. The index of the name corresponds to the variable it belongs to, for example: ["medium", "low"], where "medium" belongs to the first variable in *inputs* and "low" corresponds to the second variable in *inputs*.
- <b>Operator</b>: "and" or "or", let's choose "and".
- <b>Consequent</b>: a string corresponding one of the membership functions of your output variable, for example "high".

These three variables would then compose the rule "IF income is medium AND quality is low THEN money is high."

Complete the *calculate_firing_strength()*, that should function and check your answers by running the test statements.

In [12]:
class Rule:
    """Fuzzy rule class, initialized with an antecedent (list of strings),
    operator (string) and consequent (string)."""
    def __init__(self, n, antecedent, operator, consequent):
        self.number = n
        self.antecedent = antecedent
        self.operator = operator
        self.consequent = consequent
        self.firing_strength = 0

    def calculate_firing_strength(self, datapoint, inputs):
        # choosen min operator for T-norm
        i = 0
        self.firing_strength = 9999999999999999999
        for input_ms in self.antecedent:
            # print (i, input_ms, datapoint[i])
            mslijst = inputs[i].calculate_memberships(datapoint[i])
            msvalue = mslijst[input_ms]
            self.firing_strength = min(self.firing_strength, msvalue)
            # print (msvalue, self.firing_strength)
            i += 1
        # zou bovenstaand ook kunnen door calculate_memberships of vector van waarden te doen ???
        return self.firing_strength

In [13]:
# Test your implementation by checking the following statements
# Enter your answers in the Google form to check them, round to two decimals

rule1 = Rule(1, ["low", "amazing"], "and", "low")
print(rule1.calculate_firing_strength([200, 6.5], inputs))
print(rule1.calculate_firing_strength([0, 10], inputs))

rule2 = Rule(2, ["high", "bad"], "and", "high")
print(rule2.calculate_firing_strength([100, 8], inputs))
print(rule2.calculate_firing_strength([700, 3], inputs))

0.25
1.0
0
0.5


## 4.4 Fuzzy Rulebase


### Theory

A rulebase is simply a collection of all rules of the system!

### Practice

Our fuzzy rulebase is a collection of all rules. Create the following rules and initalize the fuzzy rulebase:
- IF income is low AND quality is amazing THEN money is low
- IF income is medium AND quality is amazing THEN money is low
- IF income is high AND quality is amazing THEN money is low
- IF income is low AND quality is okay THEN money is low
- IF income is medium AND quality is okay THEN money is medium
- IF income is high AND quality is okay THEN money is medium
- IF income is low AND quality is bad THEN money is low
- IF income is medium AND quality is bad THEN money is medium
- IF income is high AND quality is bad THEN money is high

Implement the *calculate_firing_strengths()* function that collects the highest firing strength found per membership function of the output variable in a dictionary or Counter object.
For example, if the firing strengths for the rules listed above are 0, 0, 0, 0.5, 0.25, 0, 0, 0, 0 the result would look like this: *{"low":0.5, "medium":0.25, "high"0}*.

Check the correctness of your function with the testing statements.

In [14]:
from collections import Counter

class Rulebase:
    """The fuzzy rulebase collects all rules for the FLS, can
    calculate the firing strengths of its rules."""
    def __init__(self, rules):
        self.rules = rules

    def calculate_firing_strengths(self, datapoint, inputs):
        result = Counter()
        for i, rule in enumerate(self.rules):
            fs = rule.calculate_firing_strength(datapoint, inputs)
            consequent = rule.consequent
            if fs > result[consequent]:
                result[consequent] = fs
        return result

In [15]:
# Add the rules listed in the question description
# Your code here
rule1 = Rule(1, ["low", "amazing"], "and", "low")
rule2 = Rule(2, ["medium", "amazing"], "and", "low")
rule3 = Rule(3, ["high", "amazing"], "and", "low")
rule4 = Rule(4, ["low", "okay"], "and", "low")
rule5 = Rule(5, ["medium", "okay"], "and", "medium")
rule6 = Rule(6, ["high", "okay"], "and", "medium")
rule7 = Rule(7, ["low", "bad"], "and", "low")
rule8 = Rule(8, ["medium", "bad"], "and", "medium")
rule9 = Rule(9, ["high", "bad"], "and", "high")

# print('testfs: ', rule1.calculate_firing_strength([234, 7.5], inputs))

rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9]

rulebase = Rulebase(rules)

In [16]:
# Test your implementation of calculate_firing_strengths()
# Enter your answers in the Google form to check them, round to two decimals

datapoint = [500, 3]
print(rulebase.calculate_firing_strengths(datapoint, inputs))

datapoint = [234, 7.5]
print(rulebase.calculate_firing_strengths(datapoint, inputs))

Counter({'medium': 0.5})
Counter({'low': 0.75, 'medium': 0.11333333333333333})


## 4.5. Inference (aggregation and defuzzification)

### Theory

In the Fuzzy Inference all parts of the fuzzy system come together: we are mapping an input to an output in the following way:
1. Fuzzify the input.
2. Calculate the firing strengths for the rules.
3. Use the firing strength to determine the contribution of the consequent.
4. Aggregate / collect all consequents.
5. Defuzzify

As already mentioned, there is Mamdani type inference and Takagi-Sugeno-Kang (TSK) type inference:
- With Mamdani type inference we represent the consequents of fuzzy rules as fuzzy sets (using membership functions). We use a rule's firing strength to adapt the height of the membership function in the consequent, using the implication operator (minimum or product). The consequents are then aggregated into one area (taking the maximum of all consequents for the entire input range), on which we apply a defuzzification method, such as 'largest of max', 'smallest of max' or 'centroid'.
- With TSK type inference we represent the consequents as a function of the input variables, or a constant. To combine the consequents into one output number we calculate a weighted average, where the weights are the rules' firing strengths.

<img src="https://i.imgur.com/q5lzbsZ.png"></img>
<img src="https://i.imgur.com/Yl20dJL.png"></img>

In the following image multiple defuzzification methods are visualized:
<img src="http://access.feld.cvut.cz/storage/201208252026_obr-15.png"></img>

### Practice

We will finalize our system using Mamdani type inference by performing the following three steps:
1. Gathering the largest firing strength per membership function of the output variable (implemented in your rulebase in Step 3)
2. Discretizing the range of your output variable and applying the aggregation method (<b>max</b>): for every bin you find the maximum fuzzy membership value. Notice that the membership functions of the output variable are `cut off' according to the firing strengths, with the implication method (<b>min</b>).
To accomplish this we perform two steps:
    - First we find where the aggregated area starts and ends on the x-axis
    - Second we discretize the area between start and end into 201 points (thus representing the area in 200 bins)
3. Applying two defuzzification methods: implement smallest of max (<b>som</b>) and largest of max (<b>lom</b>).

In [17]:
class Reasoner:
    def __init__(self, rulebase, inputs, output, n_points, defuzzification):
        self.rulebase = rulebase
        self.inputs = inputs
        self.output = output
        self.discretize = n_points
        self.defuzzification = defuzzification

    def inference(self, datapoint):
        # 1. Calculate the highest firing strength found in the rules per 
        # membership function of the output variable
        # looks like: {"low":0.5, "medium":0.25, "high":0}
        firing_strengths = rulebase.calculate_firing_strengths(datapoint, inputs)

        # 2. Aggragate and discretize
        # looks like: [(0.0, 1), (1.2437810945273631, 1), (2.4875621890547261, 1), (3.7313432835820892, 1), ...]
        input_value_pairs = self.aggregate(firing_strengths)

        # 3. Defuzzify
        # looks like a scalar
        crisp_output = self.defuzzify(input_value_pairs)
        return crisp_output

    def aggregate(self, firing_strengths):  
        # First find where the aggrageted area starts and ends
        # Your code here
        agg_start = self.output.range[0]
        agg_end = self.output.range[1]
        
        # Second discretize this area and aggragate
        aantal = self.discretize
        breedte = (agg_end - agg_start)/(aantal-1)
        # print(aantal, 'breedte: ', breedte)
        input_value_pairs = []
        for n in range(aantal):
            x = agg_start + n * breedte
            mslijst = self.output.calculate_memberships(x)
            # print('x:', x), print('mslijst: ', mslijst)
            # print('fs: ', firing_strengths)
            value = 0
            for ms in mslijst: 
                ms_min = min(mslijst[ms], firing_strengths[ms])
                value = max(ms_min, value)
                # print(ms_min, value)
            # print(value)
            input_value_pairs.append((x, value))
        return input_value_pairs

    def defuzzify(self, input_value_pairs):
        maxms = 0
        crisp_value = 9999
        if self.defuzzification =="som":    
            for value_pair in input_value_pairs:
                if value_pair[1]>maxms:
                    maxms = value_pair[1]
                    crisp_value = value_pair[0]
        elif self.defuzzification == "lom":
            for value_pair in input_value_pairs:
                if value_pair[1]>=maxms:
                    maxms = value_pair[1]
                    crisp_value = value_pair[0]
        # crisp_value = 9999 is eigenlijk foutsituatie
        return crisp_value

In [18]:
# Test your implementation of the fuzzy inference
# Enter your answers in the Google form to check them, round to two decimals

thinker = Reasoner(rulebase, inputs, output, 201, "som")
datapoint = [100, 1]
# firing_strengths = rulebase.calculate_firing_strengths(datapoint, inputs)
# print("fs(100,1):", firing_strengths)
# print(thinker.aggregate(firing_strengths))
print(round(thinker.inference(datapoint)))

thinker = Reasoner(rulebase, inputs, output, 101, "lom")
datapoint = [550, 4.5]
# firing_strengths = rulebase.calculate_firing_strengths(datapoint, inputs)
# print("fs(550,4.5):", firing_strengths)
print(round(thinker.inference(datapoint)))

thinker = Reasoner(rulebase, inputs, output, 201, "som")
datapoint = [900, 6.5]
# firing_strengths = rulebase.calculate_firing_strengths(datapoint, inputs)
# print("fs(900,6.5):", firing_strengths)
print(round(thinker.inference(datapoint)))

thinker = Reasoner(rulebase, inputs, output, 201, "lom")
datapoint = [100, 1]
print(round(thinker.inference(datapoint)))

thinker = Reasoner(rulebase, inputs, output, 101, "som")
datapoint = [550, 4.5]
print(round(thinker.inference(datapoint)))

thinker = Reasoner(rulebase, inputs, output, 201, "lom")
datapoint = [900, 6.5]
print(round(thinker.inference(datapoint)))

0
265
200
100
235
300
